## ■ GAN을 이용하여 일반사진을 고흐 작품스타일로 변형해보기

In [ ]:
# keras에 필요한 함수들을 불러오는 부분.
from keras.preprocessing.image import load_img, img_to_array, save_img

# 변환하려는 이미지 경로(일반 사진 경로)
target_image_path = './data/neural_style_transfer/tubingen.jpg'

# 스타일 이미지 경로(미술 사진 경로와 사진이름)
style_reference_image_path = './data/neural_style_transfer/starry-night.jpg'

# 일반 사진의 넓이와 높이 사이즈를 가져온다.
width, height = load_img(target_image_path).size

# 합성될 사진의 높이와 넓이를 설정한다.
img_height = 600
img_width = int(width * img_height / height)

print(img_width, img_height)
# 800, 600

In [ ]:
# vgg19 신경망을 import 한다.
import numpy as np
from keras.applications import vgg19 

# 일반 이미지를 vgg19 신경망에 넣기 전에 숫자로 변환하는 함수
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 미술사진 이미지로 스타일이 변환이 되면 픽셀 숫자가 -1 ~ 1사이의 
# 숫자가 되는 이를 다시 원래 픽셀인 0~255 사이의 픽셀로 변환하는 함수.
    
def deprocess_image(x):
    # ImageNet의 평균 픽셀 값을 더합니다
    x[:, :, 0] += 103.939 # red 부분에 숫자를 더함으로써 미세조정
    x[:, :, 1] += 116.779 # green
    x[:, :, 2] += 123.68 # blue
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
# keras에서 텐서플로우 함수를 사용하려고 import를 함.
from keras import backend as K

# target_image (일반 사진)와 style_reference_image(미술사진)을 
# 텐서 플로우가 인식할 수 있도록 설정
target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

# 생성된 이미지를 담을 플레이스홀더
# 합성된 이미지가 저장될 변수 (placeholder)
combination_image = K.placeholder((1, img_height, img_width, 3))

# 세 개의 이미지를 하나의 배치로 합칩니다
input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

# (배치갯수, W, H, ch)
    
# 세 이미지의 배치를 입력으로 받는 VGG 네트워크를 만듭니다.
# 이 모델은 사전 훈련된 ImageNet 가중치를 로드합니다
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', # imagenet를 학습시켜놓은 가중치를 세팅하겠다.
                    include_top=False) # 마지막 가중치를 쓰지 않겠다.

In [ ]:
# 컨텐츠 손실을 계산하는 함수
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# 스타일 손실 함수에서 사용할 그람행렬 함수
# 그람 행렬이란 이미지 특성간의 상관관계를 나타내늘 때 사용하는 행렬

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 딥러닝 게시판에 스타일 로스(style loss) 설명할 때 사용할 그림
    
# 스타일(미술작품 이미지)로 유지하기 위한 손실이 어떻게 되는지 계산을 하는 함수. 
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

# 합성된 이미지에 잡음을 줄이기 위한 함수.
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [ ]:
# vgg19의 층 이름과 활성화 텐서를 매핑(mapping) 한 딕셔너리
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# 콘텐츠 손실에 사용할 층(14층)
content_layer = 'block5_conv2'
# 스타일 손실에 사용할 층
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

# 손실 항목의 가중치 평균에 사용할 가중치 (입맛에 맞게 파라미터를 조절해줘야 됨.)
# 여기숫자에 따라 스타일에 가까워지거나 일반 사진에 가까워 지게 됨.

total_variation_weight = 1
style_weight = 100 # 이 값을 크게 주면 미술 작품 스타일이 더 살아나고
                   # 일반 사진 이미지는 좀 상쇄된다.
content_weight = 20 # 이 값을 크게 주면 일반사진 스타일이 더 살아나고
                    # 미술 작품 스타일이 상쇄된다.
                    

# 모든 손실 요소를 더해 하나의 스칼라 변수로 손실을 정의합니다.
# 1. 컨텐츠 손실, 2. 스타일 손실, 3. 합성 이미지 손실

loss = K.variable(0.) 
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :] # 일반 이미지 피쳐맵
combination_features = layer_features[2, :, :, :] # 합성 이미지 피쳐맵

loss += content_weight * content_loss(target_image_features,
                                      combination_features)


# 스타일의 loss를 전체 loss에 합쳐주는 부분
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

In [ ]:
# 손실에 대한 생성된 이미지의 그래디언트를 구합니다
grads = K.gradients(loss, combination_image)[0]

# 현재 손실과 그래디언트의 값을 추출하는 케라스 Function 객체입니다
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

# loss와 grad(기울기)를 보기 위해서 class를 생성한다.
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator() # 객체화

In [ ]:
# 사이파이의 옵티마이져 (함수의 최소값을 찾을때  bfgs 를 쓴다. )
from scipy.optimize import fmin_l_bfgs_b

result_file = './data/neural_style_transfer/style_transfer_result.png'
iterations = 1000

# 뉴럴 스타일 트랜스퍼의 손실을 최소화하기 위해 생성된 이미지에 대해 
# L-BFGS 최적화를 수행합니다
# 초기 값은 타깃 이미지입니다
# scipy.optimize.fmin_l_bfgs_b 함수가 벡터만 처리할 수 있기 때문에 이미지를 펼칩니다.

x = preprocess_image(target_image_path)
x = x.flatten()

for i in range(iterations):
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,  # x 가 바뀜에 따라 loss 가 어떻게 바뀐는지 
                                     fprime=evaluator.grads, maxfun=20)
    if i % 100 == 0:
        print('.', end=' ')
        print('현재 손실 값:', min_val)

# 생성된 현재 이미지를 저장합니다
img = x.copy().reshape((img_height, img_width, 3)) 
img = deprocess_image(img)

save_img(result_file, img)